In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import astropy
import astropy.table as atpy
from astropy import cosmology
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.table import Column

import sherpa
import sherpa.ui as ui

import scipy
import scipy.integrate
import scipy.optimize as op


import time

import emcee
import corner


#add in all needed modules for things here...

failed to import sherpa.image.ds9_backend due to 
'RuntimeErr: DS9Win unusable: Could not find ds9 on your PATH'


In [2]:
import sys
sys.tracebacklimit = 100

In [3]:
#default parameters and unit conversion factors
import defaultparams.params as params
import defaultparams.uconv as uconv

#functions to read data into format used by module
from massmod.set_prof_data import set_ne, set_tspec, set_meta

#function to fit the gas density profile
from massmod.fit_density import fitne, find_nemodeltype

#function to determine mass profile through backwards modelling
from massmod.fit_temperature import fit_ml, fit_mcmc

#analyze the marginalized posterior distribution
from massmod.posterior_mcmc import calc_posterior_mcmc, samples_results

#plotting functions
from massmod.plotting import plt_mcmc_freeparam, plt_summary, plt_summary_nice

#functions specifically to generate mock data from Vikhlinin+ profiles
from exampledata.vikh_prof import vikh_tprof, vikh_neprof, gen_vik_data

# Goal:
The primary goal of this example script is to showcase the tools available in the massmod package using mock data. The mock data is produced by randomly sampling the density and temperature profiles models published in Vikhlinin+06 for a sample of clusters. A secondary goal of this example is thus to also explore how the backwards mass modeling process used in the massmod package compares to the forward fitting results of Vikhlinin+06. The mock profiles allow for a flexible choice in noise and radial sampling rate, which allows for exploration of how these quantities affect the output of the backwards-fitting process. There is also some flexibility built into the massmod package that can be additionally tested here such as allowing for the stellar mass of the central galaxy to be included (or not included) in the model of total gravitating mass. If the stellar mass profile of the BCG is toggled on, the values for the BCG effective radius Re are pulled from the 2MASS catalog values for a de Vaucouleurs fit to their K-band data.
    
After generating the mock temperature and density profiles, the below code walks the user through fitting a model to the gas density profile, and performing the backwards-fitting mass modelling analysis. The output includes a non-parametric model fit to the temperature profile, the total mass profile and its associated parameters describing the profile, and the contributions of different mass components (i.e., DM, stars, gas) to the total mass profile.

  # A note on usage:
Any of the clusters in Vikhlinin+06 are options to be used to generate randomly sampled temperature and density profiles. The full list of clusters is as follows:
    
vikhlinin_clusters=[A133,
                    A262,
                    A383,
                    A478,
                    A907,
                    A1413,
                    A1795,
                    A1991,
                    A2029,
                    A2390,
                    RXJ1159+5531,
                    MKW4,
                    USGCS152]  
After selecting one of these clusters, this example script will automatically generate the cluster and profile data in the proper format to be used by the module. If you have your own data you would like to analyze with the massmod package, please see the included template.py file for instructions.
    

In [18]:
#select any cluster ID from the Vikhlinin+ paper
clusterID='A133'

# 1. Generate mock gas density and temperature profiles 
- according to the modeles in Vikhlinin+06. Some pertinent details are included below, more details are included in the docstrings of the functions.

Args:  
N_ne: the number of gas density profile data points  
N_temp: the number of temperature profile  
noise_ne: the percent noise on the density values  
noise_temp: the percent noise on the temperature values  
incl_mstar: include stellar mass of the central galaxy in the model for total gravitating mass  
incl_mgas: include gas mass of ICM in the model for total gravitating mass  
    
Returns:  
clustermeta: dictionary that stores relevant properties of cluster (e.g. redshift) as well as selections for analysis 
                 (e.g., incl_mstar, incl_mgas)  
ne_data: dictionary that stores the mock "observed" gas density profile  
tspec_data: dictionary that store the mock "observed" temperature profile  
nemodel_vikh: parameters of Vikhlinin+06 density model  
tmodel_vikh: parameters of Vikhlinin+06 temperature model  

In [19]:
clustermeta, ne_data, tspec_data, nemodel_vikh, tmodel_vikh= gen_vik_data(clusterID=clusterID, 
                         N_ne=50,  
                         N_temp=10,  
                         noise_ne=0.01, 
                         noise_temp=0.05,  
                         incl_mstar=1, 
                         incl_mgas=1) 


In [20]:
clustermeta

{'bcg_re': 14.22,
 'bcg_sersic_n': 4.0,
 'incl_mgas': 1,
 'incl_mstar': 1,
 'name': 'A133',
 'refindex': -1,
 'z': 0.0569}

In [21]:
ne_data[:5]

radius,ne,ne_err,radius_lowerbound,radius_upperbound
float64,float64,float64,float64,float64
39.99999999999999,0.009862023559997046,9.859983611884328e-05,0.0,0.0
42.521798377975664,0.008994138241878294,8.980562182159412e-05,0.0,0.0
45.20258343243033,0.0082622043245105,8.176621565589442e-05,0.0,0.0
48.052378471935654,0.0074876358828186535,7.455489187007704e-05,0.0,0.0
51.08183872414564,0.006759878888340406,6.81986379540131e-05,0.0,0.0


In [12]:
tspec_data

radius,tspec,tspec_err,tspec_lowerbound,tspec_upperbound,radius_lowerbound,radius_upperbound
float64,float64,float64,float64,float64,float64,float64
10.0,1.336412760425945,0.0734738494052246,0.0734738494052246,0.0734738494052246,0.0,0.0
16.272506099369238,1.6589432911346005,0.08541287493043,0.08541287493043,0.08541287493043,0.0,0.0
26.479445475400922,2.2022403308981073,0.10874812968749435,0.10874812968749435,0.10874812968749435,0.0,0.0
43.08869380063769,2.363019174770201,0.11547502375098014,0.11547502375098014,0.11547502375098014,0.0,0.0
70.11610326847304,2.3303035940840213,0.11711832895754833,0.11711832895754833,0.11711832895754833,0.0,0.0
114.0964718100231,2.231704514808246,0.11821853298428309,0.11821853298428309,0.11821853298428309,0.0,0.0
185.66355334451134,2.2303619872883536,0.11839851728314375,0.11839851728314375,0.11839851728314375,0.0,0.0
302.1211304229127,2.0954294722433797,0.11068739891884455,0.11068739891884455,0.11068739891884455,0.0,0.0
491.6267937555176,1.7385516105447176,0.08079149403924629,0.08079149403924629,0.08079149403924629,0.0,0.0


# 2. Fit the gas density profile with a parametric model

Determine the best fitting model to the density profile. Output will be one of the following: 'single_beta', 'cusped_beta', 'double_beta', 'double_beta_tied'

In [22]:
nemodeltype=find_nemodeltype(ne_data=ne_data, tspec_data=tspec_data)

'usermodel.doublebeta1d'
for dataset 1
Dataset               = 1
Method                = levmar
Statistic             = chi2
Initial fit statistic = 374271
Final fit statistic   = 2257.78 at function evaluation 38
Data points           = 50
Degrees of freedom    = 47
Probability [Q-value] = 0
Reduced statistic     = 48.0379
Change in statistic   = 372013
   beta1d.ne0     0.011562    
   beta1d.rc      50.8241     
   beta1d.beta    0.514704    
beta1d.rc lower bound:	-0.608512
beta1d.beta lower bound:	-0.00131403
beta1d.rc upper bound:	0.608512
beta1d.ne0 lower bound:	-0.000125865
beta1d.beta upper bound:	0.00131403
beta1d.ne0 upper bound:	0.000129861
Dataset               = 1
Confidence Method     = confidence
Iterative Fit Method  = None
Fitting Method        = levmar
Statistic             = chi2gehrels
confidence 1-sigma (68.2689%) bounds:
   Param            Best-Fit  Lower Bound  Upper Bound
   -----            --------  -----------  -----------
   beta1d.ne0       0.011562 -0.00

In [23]:
#best fitting model result:
print nemodeltype

double_beta_tied


In [24]:
#Find the parameters and param errors of the best-fitting gas density model
nemodel=fitne(ne_data=ne_data,tspec_data=tspec_data,nemodeltype=str(nemodeltype)) #[cm^-3]

'usermodel.doublebeta1d_tied'
for dataset 1
Dataset               = 1
Method                = levmar
Statistic             = chi2
Initial fit statistic = 447331
Final fit statistic   = 179.824 at function evaluation 745
Data points           = 50
Degrees of freedom    = 45
Probability [Q-value] = 4.99341e-18
Reduced statistic     = 3.9961
Change in statistic   = 447151
   doublebeta1d_tied.ne01   0.986196    
   doublebeta1d_tied.rc1   2.77135     
   doublebeta1d_tied.beta1   0.614747    
   doublebeta1d_tied.ne02   0.00257814  
   doublebeta1d_tied.rc2   154.237     
doublebeta1d_tied.ne01 lower bound:	-0.429997
doublebeta1d_tied.ne01 upper bound:	-----
doublebeta1d_tied.rc2 lower bound:	-2.26785
doublebeta1d_tied.rc2 upper bound:	2.26124
doublebeta1d_tied.rc1 lower bound:	-0.0316762
doublebeta1d_tied.rc1 upper bound:	1.02732
doublebeta1d_tied.beta1 lower bound:	-0.003434
doublebeta1d_tied.beta1 upper bound:	0.00346104
doublebeta1d_tied.ne02 lower bound:	-3.44852e-05
doublebeta1d_tie

In [25]:
#nemodel stores all the useful information from the fit to the gas denisty profile
print nemodel.keys()

['parmins', 'nefit', 'dof', 'parmaxes', 'rchisq', 'chisq', 'parvals', 'parnames', 'type']


# 3. Maximum likelihood estimation of free params in mass profile model

Perform the backwards-fit of the mass model. The free parameters in the fit are:
- the mass concentration "c" of the NFW profile used to model the DM halo, 
- the scale radius "Rs" of the NFW profile
- optionally, the normalization of the Sersic model $\rho_{\star,0}$ used to model the stellar mass profile of the central galaxy

In [26]:
ml_results=fit_ml(ne_data,tspec_data,nemodel,clustermeta)

scipy.optimize results
ML: c= 2.4846889455974868
ML: rs= 403.1940588304338
ML: normsersic= 6.824269206898088


these maximum likelihood results are now going to be used to initialize the walkers in our Markov chain...

# 4. Use the MCMC algorithm to determine the best fitting values and errors of the mass profile model 

The backwards-fitting mass modelling process is performed using the MCMC algorithm emcee. The walkers of the ensemble are started from the parameter estimation output by the maximum likelihood analysis. Note the number of cores the MCMC analysis is run on is an option here. 

**warning: default Nwalkers, Nsteps, Nburnin are small numbers to allow for fast testing

Returns:  
samples - the marginalized posterior distribution  
sampler - the sampler class output by emcee  

In [14]:
#fit for the mass model and temperature profile model through MCMC
samples, sampler = fit_mcmc(ne_data=ne_data, 
                            tspec_data=tspec_data, 
                            nemodel=nemodel, 
                            ml_results=ml_results, 
                            clustermeta=clustermeta,
                            Ncores=params.Ncores,
                            Nwalkers=params.Nwalkers,
                            Nsteps=params.Nsteps,
                            Nburnin=params.Nburnin)

9.354170073102437e-05
9.966820344504085e-05
9.452310626100957e-05
acceptance rate of walkers:
[0.76666667 0.53333333 0.83333333 0.76666667 0.83333333 0.8
 0.8        0.83333333 0.7        0.76666667 0.8        0.7
 0.76666667 0.7        0.8        0.7        0.76666667 0.63333333
 0.66666667 0.73333333 0.7        0.83333333 0.83333333 0.6
 0.76666667 0.8        0.9        0.73333333 0.8        0.8       ]

autocorrelation time:

AutocorrError: The chain is too short

In [17]:
print sampler

NameError: name 'sampler' is not defined

In [15]:
#analyze the marginalized MCMC distribution to calculate Rdelta, Mdelta
samples_aux = calc_posterior_mcmc(samples=samples, 
                                  nemodel=nemodel, 
                                  clustermeta=clustermeta)

NameError: name 'samples' is not defined